# Phase 3 — Feature Engineering (Titanic)
# etape 1

Objectif : transformer les données brutes en variables numériques propres
et exploitables pour la modélisation.

In [4]:
import pandas as pd

DATA_PATH = r"C:/Users/junio/OneDrive/Documents/PythonLearning/titanic_project/data/train.csv"

df = pd.read_csv(DATA_PATH)

df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
df.isna().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## Étape 2 — Sélection des colonnes

Colonnes conservées :
- Survived : variable cible.
- Pclass : proxy du statut socio-économique.
- Sex : variable fortement discriminante.
- Age : variable explicative clé (malgré valeurs manquantes).
- SibSp : structure familiale.
- Parch : structure familiale.
- Fare : indicateur économique.
- Embarked : port d’embarquement, variable catégorielle.

Colonnes supprimées :
- PassengerId : identifiant technique.
- Name : texte brut (feature engineering possible plus tard).
- Ticket : identifiant hétérogène.
- Cabin : trop de valeurs manquantes (~77%).


In [7]:
cols_to_keep = [
    "Survived",
    "Pclass",
    "Sex",
    "Age",
    "SibSp",
    "Parch",
    "Fare",
    "Embarked"
]

df = df[cols_to_keep]

df.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


## Étape 3 — Traitement des valeurs manquantes

- Age : imputation par la médiane (robuste aux valeurs extrêmes, évite de tirer la moyenne).
- Embarked : imputation par la modalité la plus fréquente (très peu de valeurs manquantes, variable catégorielle).
- Vérification : aucune valeur manquante restante après traitement.


In [ ]:
# Vérifier les NaN AVANT (Code)
df.isna().sum()


Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [10]:
age_median = df["Age"].median()
df["Age"] = df["Age"].fillna(age_median)

age_median


28.0

In [11]:
embarked_mode = df["Embarked"].mode()[0]
df["Embarked"] = df["Embarked"].fillna(embarked_mode)

embarked_mode


'S'

In [ ]:
# Vérifier les NaN APRÈS (Code)
df.isna().sum()


Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

## Étape 4 — Encodage des variables catégorielles

- Sex : encodage manuel (male=0, female=1) car variable binaire simple.
- Embarked : 
    
    * dans cette colonne avec la methode manuel et forte, on ne pouvais pas
    distinguer l'ordres de chaque valeurs distinct par consequent :
    
    one-hot encoding avec `get_dummies()` car variable nominale (pas d’ordre naturel).
    
- Pclass : conservée en numérique car variable ordinale (1re/2e/3e classe), l’ordre a du sens.


In [13]:

df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

df["Sex"].value_counts()


Sex
0    577
1    314
Name: count, dtype: int64

In [ ]:
df = pd.get_dummies(df, columns=["Embarked"], drop_first=True)

df.head()
# drop_first=True évite la redondance (dummy trap).
#Tu devrais obtenir des colonnes comme :
    #Embarked_Q
    #Embarked_S
    #(Cherbourg devient la catégorie de référence implicite)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0,3,0,22.0,1,0,7.2500,False,True
1,1,1,1,38.0,1,0,71.2833,False,False
2,1,3,1,26.0,0,0,7.9250,False,True
3,1,1,1,35.0,1,0,53.1000,False,True
4,0,3,0,35.0,0,0,8.0500,False,True


In [15]:
df.dtypes


Survived        int64
Pclass          int64
Sex             int64
Age           float64
SibSp           int64
Parch           int64
Fare          float64
Embarked_Q       bool
Embarked_S       bool
dtype: object

## Étape 5 — Création de nouvelles variables

Objectif : enrichir le dataset avec des variables dérivées basées sur une intuition métier.

Variables créées :
- **FamilySize** : taille du groupe familial à bord.
- **IsAlone** : indicateur binaire (voyage seul ou non).
- **FarePerPerson** : proxy du niveau économique individuel (prix par personne).


In [16]:
# FamilySize : nombre total de personnes dans le groupe familial
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1

# IsAlone : 1 si le passager est seul, sinon 0
df["IsAlone"] = (df["FamilySize"] == 1).astype(int)

# FarePerPerson : prix par personne
df["FarePerPerson"] = df["Fare"] / df["FamilySize"]

# Vérification rapide
df[["SibSp", "Parch", "FamilySize", "IsAlone", "Fare", "FarePerPerson"]].head()


,SibSp,Parch,FamilySize,IsAlone,Fare,FarePerPerson
0,1,0,2,0,7.2500,3.62500
1,1,0,2,0,71.2833,35.64165
2,0,0,1,1,7.9250,7.92500
3,1,0,2,0,53.1000,26.55000
4,0,0,1,1,8.0500,8.05000


In [17]:
# Vérifier qu'il ne reste aucun NaN après toutes les étapes
df.isna().sum()


Survived         0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Embarked_Q       0
Embarked_S       0
FamilySize       0
IsAlone          0
FarePerPerson    0
dtype: int64

In [ ]:
# creation de variable explicative sans la colonne survie que l'on doit expliquer
# surppression avec drop
X = df.drop("Survived", axis=1)
# colonne survie seule
y = df["Survived"]

print("Shape X:", X.shape)
print("Shape y:", y.shape)

X.head(), y.head()


Shape X: (891, 11)
Shape y: (891,)


(   Pclass  Sex   Age  SibSp  Parch     Fare  Embarked_Q  Embarked_S  \
 0       3    0  22.0      1      0   7.2500       False        True   
 1       1    1  38.0      1      0  71.2833       False       False   
 2       3    1  26.0      0      0   7.9250       False        True   
 3       1    1  35.0      1      0  53.1000       False        True   
 4       3    0  35.0      0      0   8.0500       False        True   
 
    FamilySize  IsAlone  FarePerPerson  
 0           2        0        3.62500  
 1           2        0       35.64165  
 2           1        1        7.92500  
 3           2        0       26.55000  
 4           1        1        8.05000  ,
 0    0
 1    1
 2    1
 3    1
 4    0
 Name: Survived, dtype: int64)

In [19]:
X.dtypes


Pclass             int64
Sex                int64
Age              float64
SibSp              int64
Parch              int64
Fare             float64
Embarked_Q          bool
Embarked_S          bool
FamilySize         int64
IsAlone            int32
FarePerPerson    float64
dtype: object

### Justifications métier

- ** FamilySize** : voyager en groupe peut augmenter l’entraide ou au contraire compliquer l’évacuation.
- ** IsAlone** : les passagers seuls peuvent être défavorisés sans soutien.
- ** FarePerPerson** : le prix total du billet peut refléter plusieurs personnes ; le prix par personne est un proxy plus fin du niveau économique.


## Export des données pour la phase de modélisation

À l’issue de la phase de feature engineering, le dataset est prêt
pour être utilisé par un modèle de machine learning.

Dans cette étape :
- les variables explicatives sont regroupées dans un dataset `X` ;
- la variable cible `Survived` est isolée dans un dataset `y` ;
- les deux datasets sont exportés au format CSV afin d’être réutilisés
  dans un script Python dédié à la modélisation (Phase 4).

Cette séparation permet :
- d’éviter toute fuite d’information ;
- de garantir la reproductibilité du pipeline ;
- de séparer clairement la phase d’exploration de la phase d’entraînement du modèle.


In [20]:
# Séparation des variables explicatives (X) et de la cible (y)
X = df.drop("Survived", axis=1)
y = df["Survived"]

# Vérifications rapides
print("Shape X :", X.shape)
print("Shape y :", y.shape)

# Export des datasets pour la phase 4
X.to_csv("C:/Users/junio/OneDrive/Documents/PythonLearning/titanic_project/data/X_train_phase3.csv", index=False)
y.to_csv("C:/Users/junio/OneDrive/Documents/PythonLearning/titanic_project/data/y_train_phase3.csv", index=False)

print("✅ Export terminé : X_train_phase3.csv et y_train_phase3.csv")


Shape X : (891, 11)
Shape y : (891,)
✅ Export terminé : X_train_phase3.csv et y_train_phase3.csv


In [21]:
import pandas as pd

sub = pd.read_csv("C:/Users/junio/OneDrive/Documents/PythonLearning/titanic_project/data/submission.csv")
print(sub.shape)                 # attendu: (418, 2)
print(sub.columns.tolist())      # attendu: ['PassengerId', 'Survived']
print(sub["Survived"].unique())  # attendu: [0, 1] (ordre quelconque)
print(sub.isna().sum())          # attendu: 0 partout


(418, 2)
['PassengerId', 'Survived']
[0 1]
PassengerId    0
Survived       0
dtype: int64
